In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import mlflow, os
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.compose import make_column_transformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder

from model_utils import get_pipeline
from mlFlow.MLflowTracker import MLflowTracker
from model_utils import run_training_with_mlflow
from data_prep import reduce_mem_usage


In [8]:
target = "TARGET"
train = reduce_mem_usage(pd.read_csv("../model/application_train_enriched.csv"))
test = reduce_mem_usage(pd.read_csv("../model/application_test_enriched.csv"))

Usage mémoire initial du DataFrame: 612.34 MB
Usage mémoire final du DataFrame: 211.44 MB
Mémoire réduite de 65.5 %
Usage mémoire initial du DataFrame: 121.24 MB
Usage mémoire final du DataFrame: 43.32 MB
Mémoire réduite de 64.3 %


In [9]:
# size of train and test
print(f"Train size: {train.shape}")

ones = train[train[target] == 1]
zeros = train[train[target] == 0].sample(n=len(ones)*3, random_state=42)
train = pd.concat([ones, zeros], axis=0).reset_index(drop=True)
print(f"Train size after balancing: {train.shape}")

Train size: (307511, 261)
Train size after balancing: (99300, 261)


In [10]:
X = train.drop(columns=[target])
y = train[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [11]:
number_col = X_train.select_dtypes(include=np.number).columns
print(f"{number_col=}")
categorical_col = X_train.select_dtypes(include=['object']).columns
print(f"{categorical_col=}")

number_col=Index(['SK_ID_CURR', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT',
       'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE',
       'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION',
       ...
       'DAYS_INSTALMENT_mean', 'DAYS_ENTRY_PAYMENT_min',
       'DAYS_ENTRY_PAYMENT_max', 'DAYS_ENTRY_PAYMENT_mean',
       'CNT_INSTALMENT_sum', 'CNT_INSTALMENT_mean', 'SK_DPD_max_y',
       'SK_DPD_mean_y', 'SK_DPD_DEF_max_y', 'SK_DPD_DEF_mean_y'],
      dtype='object', length=245)
categorical_col=Index(['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE',
       'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'HOUSETYPE_MODE',
       'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE'],
      dtype='object')


In [12]:
param_grid = {
    "model__n_estimators": [100, 200],
    "model__max_depth": [10, 20],
    "model__min_samples_split": [2, 5],
    "model__min_samples_leaf": [1, 2],
    "model__max_features": ["sqrt", "log2"],
     "model__class_weight": ['balanced', {0: 1, 1: 10}]
}

In [13]:
pipeline = get_pipeline(RandomForestClassifier(), numeric_features=number_col, categorical_features=categorical_col)
grid = GridSearchCV(pipeline, param_grid, cv=skf, n_jobs=-1, verbose=2)

In [ ]:
tracker = MLflowTracker("home_credit_experiment")

best_model, metrics = run_training_with_mlflow(
    grid=grid,
    pipeline=pipeline,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    tracker=tracker,
    run_name="rf_gridsearch_v1",
    model_name="home_credit_rf"
)


2025/12/12 18:38:14 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/12/12 18:38:14 INFO mlflow.store.db.utils: Updating database tables
2025-12-12 18:38:14 INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
2025-12-12 18:38:14 INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2025-12-12 18:38:15 INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
2025-12-12 18:38:15 INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


[MLflow] Tracking sur: sqlite:///C:/Users/corentin/Nextcloud/Onedrive-Esaip/1 Cours/S7/Majeur Projets/Project DEA/mlruns/mlflow.db
Fitting 5 folds for each of 64 candidates, totalling 320 fits


In [ ]:
y_pred = grid.predict(X_test)
print("Best parameters:")
print(grid.best_params_)

print("Accuracy :", accuracy_score(y_test, y_pred))
print("F1 Score :", f1_score(y_test, y_pred, average='weighted'))
print(classification_report(y_test, y_pred))

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt="d", cmap="Blues")
plt.title("Matrice de confusion")
plt.show()
model = grid.best_estimator_.named_steps["model"]
preprocessor = grid.best_estimator_.named_steps["preprocessing"]
feature_names = preprocessor.get_feature_names_out()
clean_names = [name.split("__")[-1] for name in feature_names]
importances = pd.Series(model.feature_importances_, index=clean_names)
importances.sort_values(ascending=False).head(25).plot(kind="barh", figsize=(12, 8))
plt.title("Top 15 Features importantes")
plt.show()

# Logistic Regression Model

In [ ]:
param_grid = {
    "model__C": [0.01, 0.1, 1, 10],
    "model__solver": ['liblinear', 'saga'],
    "model__max_iter": [200],
    "model__class_weight": ['balanced', {0: 1, 1: 10}]
}
pipeline = get_pipeline(LogisticRegression(), numeric_features=number_col, categorical_features=categorical_col)
grid = GridSearchCV(pipeline, param_grid, cv=skf, n_jobs=-1, verbose=2)
tracker = MLflowTracker("home_credit_experiment")

best_model, metrics = run_training_with_mlflow(
    grid=grid,
    pipeline=pipeline,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    tracker=tracker,
    run_name="logreg_gridsearch_v1",
    model_name="home_credit_logreg"
)

# LightGBM

In [ ]:
param_grid = {
    "model__class_weight": ['balanced', {0: 1, 1: 10}]
}
from lightgbm import LGBMClassifier
pipeline = get_pipeline(LGBMClassifier(), numeric_features=number_col, categorical_features=categorical_col)
grid = GridSearchCV(pipeline, param_grid, cv=skf, n_jobs=-1, verbose=2)
tracker = MLflowTracker("home_credit_experiment")

In [ ]:
best_model, metrics = run_training_with_mlflow(
    grid=grid,
    pipeline=pipeline,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    tracker=tracker,
    run_name="lgbm_gridsearch_v1",
    model_name="home_credit_lgbm"
)

# MLP Classifier

In [ ]:
param_grid = {
    "model__hidden_layer_sizes": [(50,), (100,), (50, 50)],
    "model__activation": ['relu', 'tanh'],
    "model__solver": ['adam'],
    "model__alpha": [0.0001, 0.001],
    "model__learning_rate": ['constant', 'adaptive'],
    "model__max_iter": [200],
}
from sklearn.neural_network import MLPClassifier
pipeline = get_pipeline(MLPClassifier(), numeric_features=number_col, categorical_features=categorical_col)
grid = GridSearchCV(pipeline, param_grid, cv=skf, n_jobs=-1, verbose=2)
tracker = MLflowTracker("home_credit_experiment")

In [ ]:
best_model, metrics = run_training_with_mlflow(
    grid=grid,
    pipeline=pipeline,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    tracker=tracker,
    run_name="mlp_gridsearch_v1",
    model_name="home_credit_mlp"
)

# Submit predictions

In [ ]:
pred = best_model.predict(test)
submission = pd.DataFrame({
    "SK_ID_CURR": test["SK_ID_CURR"],
    "TARGET": pred
})
submission.to_csv("../model/submission.csv", index=False)